# 第一章 初识强化学习

**强化学习**：生物为了趋利避害而更频繁地实施对自己有利的策略

**强化**：特定刺激使生物更趋向于采用某些策略的现象

## 强化学习及其关键元素

* 在一个强化学习系统中，决策者可以观察环境，并根据观测行动。在行动之后，就能获得奖励。
* 强化学习通过与环境的交互来学习如何来最大化奖励

**关键元素**：
* **奖励**(reward)：强化学习系统的目标，即<u>最大化长时间里的总奖励</u>
* **策略**(policy)：从观测到动作的关系称为策略。强化学习的学习对象就是策略。

**与监督学习、非监督学习的区别**：
* 相比于监督学习：RL不知道每个动作的正确答案，只能通过奖励信号学习。监督学习希望将学习的结果运用于未知数据
* 相比于非监督学习：非监督学习旨在发现数据之间的隐含结构，而强化学习有着明确的数值目标。

### 智能体/环境接口(Agent-Environment Interface)

【**相关定义**】
* **智能体**(agent)：强化学习的决策者和学习者，可做出决策，接收奖励信号
  * 不需对其本身建模
* **环境**(Environment)：除智能体外所有事物，是智能体的交互对象

**核心思想**：分隔主管可以控制的部分和客观不能改变的部分

【**主要环节**】
* **观测**(observation)：智能体观测环境，记作$O$
* **动作**(action)：智能体根据观测做出决策，决定对环境施加的动作，记作$A$
* **状态**(state)：环境受智能体动作影响，改变自己状态，记作$S$，并给出**奖励**(reward)，记作$R$

其中，O、A、R为智能体可直接观测得到的内容。

【**引入时间**】

* 很多RL问题具有时间顺序或因果顺序，此时其具有先后顺序，且先前状态和动作会对后续产生影响
* 此时引入时间指标t，分别对应$S_t$，$A_t$，$O_t$，$R_t$
  * 不是所有的RL都需要引入时间
* **时间指标离散化**：智能体和环境在离散的时间步骤上交互，此时可建模为离散时间智能体/环境接口。假设交互时间t=0,1,2,...，在时刻t发生事件顺序如下：
  * 智能体观测时间得$O_t$
  * 智能体根据观测环境决定做出动作$A_t$
  * 环境根据智能体动作，给予智能体奖励$R_{t+1}$，并进入下一步状态$S_{t+1}$


## 分类

1. 按任务分类：
   * **单智能体任务与多智能体任务**
   * **回合制任务与连续性任务**：回合制任务有明确的开始和结束状态；连续性任务则没有
   * **离散时间环境和连续时间环境**：离散时间环境下智能体和环境的交互分步进行；连续时间下进行交互则为连续时间环境
   * **离散动作空间和连续动作空间**：动作数量有限/无限
   * **确定性环境和非确定性环境**：据环境是否随机分类
   * **完全可观测环境和非完全可观测环境**：是否能观测到环境的全部知识
2. 按算法分类：
   * **同策学习和异策学习**：同策边决策边学习，学习者同时也是决策者；异策通过之前的历史学习，学习者和决策者可不同，学习者不一定要知道当时决策
   * **有模型和无模型**：有无环境的数学模型
   * **回合制更新和时序差分更新**：回合制更新是在回合结束后利用整个回合的信息更新学习；时序差分更新不需要等回合结束，可综合利用现有更新和估计学习
   * **基于价值和基于策略**：
     * 基于价值的RL定义了<u>基于动态或动作的价值函数</u>，表示达到某种策略或执行某种动作后得到的回报
     * 基于策略的RL不需要定义价值函数，它可以为动作分配概率分布，按照概率分布执行动作
   * **深度强化学习** 

## 基于Gym的智能体/环境交互

[gym](https://gym.openai.com/)是OpenAI推出的强化学习实验环境库。
具体使用如下。

列出所有环境：

In [ ]:
import numpy as np
np.random.seed(0)
import pandas as pd
import gym

In [ ]:
space_names = ['观测空间', '动作空间', '奖励范围', '最大步数']
df = pd.DataFrame(columns=space_names)

env_specs = gym.envs.registry.all()
for env_spec in env_specs:
    env_id = env_spec.id
    try:
        env = gym.make(env_id)
        observation_space = env.observation_space
        action_space = env.action_space
        reward_range = env.reward_range
        max_episode_steps = None
        if isinstance(env, gym.wrappers.time_limit.TimeLimit):
            max_episode_steps = env._max_episode_steps
        df.loc[env_id] = [observation_space, action_space, reward_range, max_episode_steps]
    except:
        pass

with pd.option_context('display.max_rows', None):
    display(df)

### 小车上山 MountainCar-v0

**环境**：Gym的MountainCar-v0

可通过`gym.make()`去制造gym的内置环境

每个环境都有自己的观测空间和动作空间，分别用`env.observation_space`、`env.action_space`表示。

【**离散空间和连续空间**】

* **离散空间**：`gym.spaces.Discrete`
  * `Discrete`类实例成员`n`表示有几个可能取值
* **连续空间**：`gym.spaces.Box`
  * `Box`类实例成员`high`和`low`表示每个浮点数取值范围

In [ ]:
env = gym.make('MountainCar-v0')
print('观测空间 = {}'.format(env.observation_space))
print('动作空间 = {}'.format(env.action_space))
print('观测范围 = {} ~ {}'.format(env.observation_space.low,
                                env.observation_space.high))
print('动作数 = {}'.format(env.action_space.n))

**智能体**：一个根据指定确定性策略决定动作并且不学习的智能体

需要自己定义类来表示智能体，最起码应该有初始化`__init__()`、以及与学习、决策相关的函数（同策）

In [ ]:
class BespokeAgent:
    def __init__(self,env):
        pass
    
    def decide(self,observation):   # 决策
        position, velocity = observation
        lb = min(-0.09 * (position + 0.25) ** 2 + 0.03,
                0.3 * (position + 0.9) ** 4 - 0.008)
        ub = -0.07 * (position + 0.38) ** 2 + 0.07
        if lb < velocity < ub:
            action = 2
        else:
            action = 0
        return action # 返回动作
    
    def learn(self,*args):      # 学习
        pass

agent = BespokeAgent(env)

**智能体与环境交互**

* 初始化环境对象`env`：`env.reset()`
* 环境对象的`step()`方法：接收**动作**作为参数，返回如下四个参数
  * 观测：np.array对象，表示观测
  * 奖励：float类型的值
  * 本回合结束指示：bool类型数值，可据此判断当前动作后游戏是否结束
  * 其他信息：dict类型数值。含调试信息，可不要
* 图形化环境：`env.render()`
* 关闭图形化环境：`env.close()`

In [ ]:
def play_montecarlo(env, agent, render=False, train=False):
    episode_reward = 0.             # 记录回合总奖励，初始化为0
    observation = env.reset()       # 重置游戏环境，开始新回合
    while True:                     # 不断循环，直到回合结束
        if render:                  # 判断是否显示
            env.render()            # 显示图形界面，图形界面可以用 env.close() 语句关闭
        action = agent.decide(observation)
        print(type(action))
        next_observation, reward, done, _ = env.step(action) # 执行动作
        episode_reward += reward    # 收集回合奖励
        if train:                   # 判断是否训练智能体
            agent.learn(observation, action, reward, done) # 学习
        if done:                    # 回合结束，跳出循环
            break
        observation = next_observation
    return episode_reward           # 返回回合总奖励

交互1回合，并显示图形化界面：

In [ ]:
env.seed(0) # 设置随机数种子,只是为了让结果可以精确复现,一般情况下可删去
episode_reward = play_montecarlo(env, agent, render=True)
print('回合奖励 = {}'.format(episode_reward))
env.close()

评估性能：交互100回合求平均

In [ ]:
episode_rewards = [play_montecarlo(env, agent) for _ in range(100)]
print('平均回合奖励 = {}'.format(np.mean(episode_rewards)))